In [1]:
import cobra
from cobra import Reaction, Metabolite, Model
import pandas as pd
import numpy

In [2]:
#loading new metabolite data
current_model = cobra.io.read_sbml_model("results/reconstructions/v4_with_all_annotations.xml")

#loading new metabolite data
meta_data = pd.read_csv("data/metabolite_ids_Mhamdi_Ma.csv")
print(meta_data)

       met_id                     met_name  source
0    cpd23643                Levoglucosan   Mhamdi
1    cpd00881                Caffeic acid   Mhamdi
2    cpd03847           Tetradecanoic acid  Mhamdi
3    cpd00033                      Glycine  Mhamdi
4    cpd00383              Shikimic acid    Mhamdi
5    cpd00373  trans ou cis Sinapinic acid  Mhamdi
6    cpd00121                Myo-inositol   Mhamdi
7    cpd00214               Palmitic acid   Mhamdi
8    cpd00139               Glycolic acid   Mhamdi
9    cpd01122               Linoleic acid   Mhamdi
10   cpd00129                     Proline   Mhamdi
11   cpd01138                Threonic acid  Mhamdi
12   cpd00069                    Tyrosine   Mhamdi
13   cpd01107              Decanoic acid    Mhamdi
14   cpd00009            Phosphoric acid    Mhamdi
15   cpd00036              Succinic acid    Mhamdi
16   cpd00132                 Asparagine    Mhamdi
17   cpd00722            O-Acetyl-L-serine  Mhamdi
18   cpd00136       p-Hydroxybe

In [4]:

#checking for metabolite in current model (cpdXXXXX_e or _c)

in_mod = []
out_mod = []

met_id = meta_data['met_id'].tolist()

for cpd_id in met_id:
    try:
        current_model.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
        in_mod.append(cpd_id)
    except:
         out_mod.append(cpd_id)
            

In [6]:
#checking if metabolite has exchange reaction
RX_in_mod = []
RX_out_mod = []

for cpd_id in in_mod:
    try:
        current_model.reactions.get_by_id("EX_"+str(cpd_id))
        RX_in_mod.append(cpd_id)
    except:
        RX_out_mod.append(cpd_id)


In [7]:
#UNNECESSARY all existing metabolites already have exchange rxns
#create exchange reaction of existing metabolites w bounds -1000,1000
for cpd_id in RX_out_mod:
    Ex_rxn = cobra.Reaction("EX_" + str(cpd_id) + "_c")
    met_obj = current_model.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
    Ex_rxn.add_metabolites({met_obj.copy():-1})

    Ex_rxn.id = "EX_"+str(cpd_id)
    Ex_rxn.lower_bound = -1000
    Ex_rxn.upper_bound = 1000
    current_model.add_reaction(Ex_rxn)
    
    

Ignoring reaction 'EX_cpd00033' since it already exists.
Ignoring reaction 'EX_cpd00033' since it already exists.
Ignoring reaction 'EX_cpd00161' since it already exists.
Ignoring reaction 'EX_cpd00020' since it already exists.
Ignoring reaction 'EX_cpd00223' since it already exists.
Ignoring reaction 'EX_cpd00169' since it already exists.
Ignoring reaction 'EX_cpd00036' since it already exists.
Ignoring reaction 'EX_cpd00106' since it already exists.
Ignoring reaction 'EX_cpd00130' since it already exists.


In [8]:
#loading universal model for meatbolites not found in current model
universal = cobra.io.load_json_model("data/modelseed_data/seed_universal.json")

In [11]:
#checking if metabolites not in model are in universal
in_uni = []
out_uni = []

for cpd_id in out_mod:
    try:
        universal.metabolites.get_by_id(str(cpd_id)+"_c" or "_e" or "")
        in_uni.append(cpd_id)
    except:
         out_uni.append(cpd_id)

print(out_uni)

['cpd23643', 'cpd30321', 'cpd23861', 'cpd19036', 'cpd19001', 'cpd05274', 'cpd00724', 'cpd19036', 'cpd19035']


In [14]:
#make copy of metabolite from universal
#create exchange reaction for universal metabolites w bounds -1000,1000
for cpd_id in in_uni:
    met_obj = universal.metabolites.get_by_id(str(cpd_id)+"_c" or "_e")
    Ex_rxn = cobra.Reaction("EX_" + str(cpd_id))
    Ex_rxn.add_metabolites({met_obj.copy():-1})
    
    Ex_rxn.id = "EX_"+str(cpd_id)
    Ex_rxn.lower_bound = -1000
    Ex_rxn.upper_bound = 1000
    current_model.add_reaction(Ex_rxn)
    


Ignoring reaction 'EX_cpd01122' since it already exists.
Ignoring reaction 'EX_cpd00059' since it already exists.
Ignoring reaction 'EX_cpd00139' since it already exists.


In [16]:
#save model with new exchange reactions
cobra.io.write_sbml_model(current_model, "v5_with_all_annotations.xml", use_fbc_package=True)